In [3]:
import jax.scipy.interpolate as interp
import jax.numpy as jnp
import numpy as np
import time

In [7]:
OM = np.random.uniform(0,1,10)
Ode = np.random.uniform(0,1,10)
w = np.random.uniform(-3,1,10)
wa = np.random.uniform(-3,1,10)
alpha = np.random.uniform(0,1,10)

def func(p0,p1,p2,p3,p4):
    return p0+p1+p2+p3+p4

param_list = {'OM':OM, 'Ode':Ode,'w':w,'wa':wa,'alpha':alpha}
points = np.array([OM, Ode])
# values = func(*[np.meshgrid(*[param_list[elem] for elem in param_list.keys()])[p_i] for p_i in range(len(param_list.keys()))])#.flatten()#jnp.array([[10, 20, 30], [40, 50, 60], [70, 80, 90]])
values = func(OM,Ode,w,wa,alpha)
# t1 = time.time()
# interpolate = interp.RegularGridInterpolator(points, values, method='linear')
# t2 = time.time()
# query_points = jnp.array(np.random.random(1,5))
# interpolate(query_points)
# t3 = time.time()
# print('Interpolation time:',t2-t1)
# print('Evaluation time:',t3-t2)


In [8]:
points.shape,values.shape

((2, 10), (10,))

array([0.83338197, 0.88922726])

In [4]:
def get_nearest_value(data,
                      value,
                      ):
    """Get the nearest value in a dataset

    Args:
        data: array of values to hunt through
        value: value to find nearest in data to

    Returns:
        Nearest value
    """

    # Find the nearest below and above
    diff = data - value
    less = np.where(diff <= 0)
    greater = np.where(diff >= 0)

    # Get the position and value for above and below. If we're at the edge but somehow this doesn't work, just
    # go for the lowest or highest values
    if len(less[0]) == 0:
        nearest_below = data[0]
    else:
        nearest_below_idx = np.argsort(np.abs(diff[less]))[0]
        nearest_below = data[less][nearest_below_idx]

    if len(greater[0]) == 0:
        nearest_above = data[-1]
    else:
        nearest_above_idx = np.argsort(diff[greater])[0]
        nearest_above = data[greater][nearest_above_idx]

    nearest_value = np.array([nearest_below, nearest_above])

    # If we're actually choosing a grid value, just return that singular value
    if np.diff(nearest_value) == 0:
        nearest_value = nearest_value[0]

    return nearest_value

In [5]:
def get_nearest_values(dataset,
                       keys,
                       values,
                       ):
    """Function to parallelise get_nearest_value

    Args:
        dataset (dict): Dataset to hunt through
        keys (list): List of keys to search with
        values (list): List of values to find in the dataset

    Returns
        list of the nearest values
    """

    nearest_values_list = [get_nearest_value(dataset[keys[i]].values, values[i])
                           for i in range(len(values))]

    return nearest_values_list

In [ ]:
from cosmology_JAX import j_r_SL
import jax_cosmo as jc
import jax.numpy as jnp

In [25]:
import numpy as np
zL_1D = np.linspace(0,5,100)
zS_1D  = np.linspace(5,10,100)
OM_1D = np.linspace(0,1,100)
Ok_1D = np.linspace(-1,1,100)
w_1D = np.linspace(0,1,100)
wa_1D = np.linspace(0,1,100)
# zL_mesh,zS_mesh,OM_mesh,Ok_mesh,w_mesh,wa_mesh = np.meshgrid(zL_1D,zS_1D,OM_1D,Ok_1D,w_1D,wa_1D)
# zL_flat = zL_mesh.flatten()
# zS_flat = zS_mesh.flatten()
# OM_flat = OM_mesh.flatten()
# Ok_flat = Ok_mesh.flatten()
# w_flat = w_mesh.flatten()
# wa_flat = wa_mesh.flatten()

def r_theory_interp(zL,zS,OM,Ok,w,wa):
    s8 = 0.8;n_s = 0.96;Ob=0;H0=70
    j_cosmo = jc.Cosmology(Omega_c=OM, h=H0/100, Omega_k=Ok, w0=w,
                         Omega_b=Ob, wa=wa, sigma8=s8, n_s=n_s)
    return j_r_SL(zL,zS,j_cosmo)

def r_theory_vectorize(zL,zS,OM,Ok,w,wa):
    return jnp.vectorize(r_theory_interp)(zL,zS,OM,Ok,w,wa)

In [26]:
from tqdm.notebook import tqdm
for OM in tqdm(OM_1D):
    for Ok in tqdm(Ok_1D):
        for w in tqdm(w_1D):
            for wa in wa_1D:
                r_flat = r_theory_interp(zL=zL_1D,
                        zS = zS_1D,
                        OM = OM,
                        Ok = Ok,
                        w = w,
                        wa = wa)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
from scipy.interpolate import RegularGridInterpolator

X = np.linspace(0,1,100000000)
Y = np.linspace(0,1,50000000)
# x_mesh,y_mesh = np.meshgrid(X,Y)
# f_mesh = x_mesh**2+y_mesh**3
f = X**2
x_true = 0.7
print('Get nearest')
nearest_values = get_nearest_value(data=X,value=x_true)
print(nearest_values)
print('np.where')
grid_coords = np.where(X==np.array([nearest_values]).T)[1]
print('coords',grid_coords)
print(grid_coords.shape, f[grid_coords].shape)
print('Interp')
reg_grid = RegularGridInterpolator((X[grid_coords],), f[grid_coords])
print('final')
reg_grid(np.array([x_true]))

Get nearest
[0.7        0.70000001]
np.where
coords [69999999 70000000]
(2,) (2,)
Interp
final


array([0.49])

In [16]:
from scipy.interpolate import RegularGridInterpolator

nearest_values = get_nearest_vals(whatever)
grid_coords = (*nearest_values)
reg_grid = RegularGridInterpolator(grid_coords, full_grid[nearest_vals])
interp_val = reg_grid(coords)

IndexError: arrays used as indices must be of integer (or boolean) type